In [1]:
# https://adventofcode.com/2021/day/6
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
t_pregnancy = 7
n_max = 9


def load_input(is_sample=False):
    if is_sample:
        ages = [3, 4, 3, 1, 2]
    else:
        ages = [3,4,3,1,2,1,5,1,1,1,1,4,1,2,1,1,2,1,1,1,3,4,4,4,1,3,2,1,3,4,1,1,3,4,2,5,5,3,3,3,5,1,4,1,2,3,1,1,1,4,1,4,1,5,3,3,1,4,1,5,1,2,2,1,1,5,5,2,5,1,1,1,1,3,1,4,1,1,1,4,1,1,1,5,2,3,5,3,4,1,1,1,1,1,2,2,1,1,1,1,1,1,5,5,1,3,3,1,2,1,3,1,5,1,1,4,1,1,2,4,1,5,1,1,3,3,3,4,2,4,1,1,5,1,1,1,1,4,4,1,1,1,3,1,1,2,1,3,1,1,1,1,5,3,3,2,2,1,4,3,3,2,1,3,3,1,2,5,1,3,5,2,2,1,1,1,1,5,1,2,1,1,3,5,4,2,3,1,1,1,4,1,3,2,1,5,4,5,1,4,5,1,3,3,5,1,2,1,1,3,3,1,5,3,1,1,1,3,2,5,5,1,1,4,2,1,2,1,1,5,5,1,4,1,1,3,1,5,2,5,3,1,5,2,2,1,1,5,1,5,1,2,1,3,1,1,1,2,3,2,1,4,1,1,1,1,5,4,1,4,5,1,4,3,4,1,1,1,1,2,5,4,1,1,3,1,2,1,1,2,1,1,1,2,1,1,1,1,1,4]
    
    return np.array(ages)  


def next_day(ages):    
    n_child = np.sum(ages == 0)
    
    ages -= 1
    ages = np.where(ages==-1, 6, ages)
    #ages = update_age(ages)
    
    if n_child > 0:
        #ages = np.append(ages, np.ones((1, n_child))*8)
        ages = np.append(ages, np.tile(8, (1, n_child)))
    
    return ages.astype(int)

# I thought that it is faster than np.where
# but it wasn't.
def update_age(ages):
    ages_ = np.copy(ages)
    ages_[ages==-1] = 6
    return ages_


def after_n_days(ages, n):
    for i in range(n):
        ages = next_day(ages)
    return ages


def after_n_days_each(n):
    breed = {}

    for i in range(7):
        ages = np.array([i])
        ages = after_n_days(ages, n) 
        breed[i] = len(ages)
    return breed


## use count, instead of real array.
def count_ages(ages):
    c = {}
    for i in range(n_max):
        c[i] = list(ages).count(i)
    return c


def next_day_count(c):    
    n_child = c[0]

    for i in range(1, n_max):
        c[i-1] = c[i]
    c[6] += n_child
    c[8] = n_child

    return c


def after_n_days_count(c, n):
    for i in range(n):
        c = next_day_count(c)
    return c


def after_n_days2(ages, breed):
    return np.sum([breed[a] for a in ages])

In [12]:
ages = load_input()
c = count_ages(ages)

## part 1
c = after_n_days_count(c, 80)
## part 2
#c = after_n_days_count(c, 256)

sum(c.values())

371379

## [experiments] can we use recursive function?

In [9]:
n = 50
ages = load_input()
#time0 = time.time()
breed = {}
time0 = time.time()
#for i in [0]:
for i in range(t_pregnancy):
    ages = np.array([i])
    ages = after_n_days(ages, n) 
    breed[i] = len(ages)
    print(f'fish {i} becomes {breed[i]} fishes.')
#    print(f'elapsed time: {time.time() - time0} [s]')

fish 0 becomes 107 fishes.
fish 1 becomes 106 fishes.
fish 2 becomes 86 fishes.
fish 3 becomes 86 fishes.
fish 4 becomes 71 fishes.
fish 5 becomes 70 fishes.
fish 6 becomes 64 fishes.


In [115]:
def get_n_child(i, n_day):
    n_child = (n_day - i)//t_pregnancy + 1
    if (n_day - i) % t_pregnancy == 0:
        n_child -= 1 
    return n_child


def calc_total(i, n_day):
    # self + child
    n_total = 1 + get_n_child(i, n_day)
    
    days_left = n_day - i - 1
    while days_left > 0:
        n_total += get_n_child(8, days_left)
        days_left -= 8
        
    return n_total

In [109]:
def after_n_days_each(days_left):
    breed = {}

    for i in range(7):
        breed[i] = calc_total(i, days_left)
        #print(f' fish {i} becomes {breed[i]} fishes.')
        #print(f'elapsed time: {time.time() - time0} [s]')
    return breed
breed = after_n_days_each(18)
breed

{0: 6, 1: 6, 2: 5, 3: 5, 4: 4, 5: 4, 6: 4}

In [34]:
def get_delivery_days(i, n_day):
    delivery_days = [day for day in range(i, n_day, t_pregnancy)]
    return delivery_days

n_day = 80
i = 3

total = 1
# how many days later new fish will be delivered.
delivery_days = get_delivery_days(i, n_day)
print(delivery_days)
total += len(delivery_days)

def next_generation(delivery_days, n_day):
    x = {}
    total = 0
    for d in delivery_days:
        delivery_days_ = get_delivery_days(8, n_day -d-1)
        print(f"{d}: {delivery_days_}")
        n_child = len(delivery_days_)

        if not n_child == 0: 
            total += n_child
            x[d] = delivery_days_
    return x, total

x, total_ = next_generation(delivery_days, n_day)
#total += total_
#total

[3, 10, 17, 24, 31, 38, 45, 52, 59, 66, 73]
3: [8, 15, 22, 29, 36, 43, 50, 57, 64, 71]
10: [8, 15, 22, 29, 36, 43, 50, 57, 64]
17: [8, 15, 22, 29, 36, 43, 50, 57]
24: [8, 15, 22, 29, 36, 43, 50]
31: [8, 15, 22, 29, 36, 43]
38: [8, 15, 22, 29, 36]
45: [8, 15, 22, 29]
52: [8, 15, 22]
59: [8, 15]
66: [8]
73: []


In [35]:
for d in x:
    #print(f"{d}: {x[d]}")
    x_, total_ = next_generation(x[d], n_day -d + 1)
    #print(x_)

8: [8, 15, 22, 29, 36, 43, 50, 57, 64]
15: [8, 15, 22, 29, 36, 43, 50, 57]
22: [8, 15, 22, 29, 36, 43, 50]
29: [8, 15, 22, 29, 36, 43]
36: [8, 15, 22, 29, 36]
43: [8, 15, 22, 29]
50: [8, 15, 22]
57: [8, 15]
64: [8]
71: []
8: [8, 15, 22, 29, 36, 43, 50, 57]
15: [8, 15, 22, 29, 36, 43, 50]
22: [8, 15, 22, 29, 36, 43]
29: [8, 15, 22, 29, 36]
36: [8, 15, 22, 29]
43: [8, 15, 22]
50: [8, 15]
57: [8]
64: []
8: [8, 15, 22, 29, 36, 43, 50]
15: [8, 15, 22, 29, 36, 43]
22: [8, 15, 22, 29, 36]
29: [8, 15, 22, 29]
36: [8, 15, 22]
43: [8, 15]
50: [8]
57: []
8: [8, 15, 22, 29, 36, 43]
15: [8, 15, 22, 29, 36]
22: [8, 15, 22, 29]
29: [8, 15, 22]
36: [8, 15]
43: [8]
50: []
8: [8, 15, 22, 29, 36]
15: [8, 15, 22, 29]
22: [8, 15, 22]
29: [8, 15]
36: [8]
43: []
8: [8, 15, 22, 29]
15: [8, 15, 22]
22: [8, 15]
29: [8]
36: []
8: [8, 15, 22]
15: [8, 15]
22: [8]
29: []
8: [8, 15]
15: [8]
22: []
8: [8]
15: []
8: []


In [166]:
def calc_total(i, n_day):
    delivery_days = get_delivery_days(i, n_day)
    print(delivery_days)
    
    if len(delivery_days) > 0:
        for j in delivery_days:
            delivery_days = get_delivery_days(8, n_day -j-1)
            print(f"{j} + {delivery_days}")

        if not n_child == 0: 
            total += n_child
    else:
        print("end.")
        return total
    
    return total

calc_total(3, 18)

[3, 10, 17]
3 + [8]
10 + []
17 + []


UnboundLocalError: local variable 'total' referenced before assignment

In [151]:
after_n_days(np.array([3]), 18)

[3]
[2]
[1]
[0]
[6 8]
[5 7]
[4 6]
[3 5]
[2 4]
[1 3]
[0 2]
[6 1 8]
[5 0 7]
[4 6 6 8]
[3 5 5 7]
[2 4 4 6]
[1 3 3 5]
[0 2 2 4]
[6 1 1 3 8]


array([6, 1, 1, 3, 8])

In [139]:
list(range(18, 3, -t_pregnancy))

[18, 11, 4]